In [ ]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    """
    Implements the sinusoidal positional encoding described in "Attention Is All You Need".
    PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()

class TransformerEncoder(nn.Module):
    """
    A simple Transformer encoder block.
    """
    def __init__(self, d_model, num_heads, vocab_size):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)

class Transformer(nn.Module):
    """
    A simple Transformer model, as in Vaswani et al. (2017).
    """
    def __init__(self, d_model, num_heads, num_layers, vocab_size):
        super().__init__()
        self.encoder = TransformerEncoder()